In [127]:
from textblob import TextBlob
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np


import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from tokenizers import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 

In [79]:
def preprocess_text(text):

    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())

    words = [word for word in words if word not in string.punctuation]

    # Remove stop words
    words = " ".join([word for word in words if word.isalpha()]).lower()    
        
    return words


In [80]:
url = "https://www.theonion.com/nickelodeon-announces-dan-schneider-has-been-chemically-1851370242"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

parsed_url = urlparse(url).netloc
base_url = parsed_url[4:]

text = []
# Find all <p> elements and print their text
for paragraph in soup.find_all('p'):
    text.append(preprocess_text(paragraph.get_text()))

In [81]:
text = [phrase for phrase in text if phrase.count(' ') > 2]
text

['los angeles saying the channel would do everything in its power to make up for the writer and producer s years of toxic abusive behavior nickelodeon announced thursday that dan schneider had been chemically castrated with slime while in his powerful position at nickelodeon mr schneider harassed child stars and cast them in sexualized scenes which is why we have elected to sterilize him with a toxic dosage of neon green slime said ceo brian robbins adding that the creator of drake josh icarly and sam cat had been tied up and had a large vat of the viscous liquid poured over his body until his testicles shriveled up and his penis fell off we sincerely regret allowing dan to spend years preying on those less powerful than him and now that the slime has permanently mangled his genitals we are confident he will never do so again rest assured the procedure was incredibly painful he will not make the same mistake again at press time nickelodeon added that schneider would be forced to carry 

### TextBlob: Determining Polarity

In [82]:
polarity_score = []

for phrase in text:
    polarity_score.append(TextBlob(phrase).sentiment.polarity)


#### polarity denotes the sentiment of text. values lie in [-1,1]. -1 denotes a highly negative sentiment and 1 denotes a hightly positive sentiment

In [83]:
polarity_min = min(polarity_score)
polarity_max = max(polarity_score)
polarity_std = np.std(polarity_score)
polarity_mean = np.mean(polarity_score)
polarity_range = polarity_max - polarity_min

In [84]:
print("Min: ",polarity_min)
print("Max: ",polarity_max)
print("Standard Deviation: ", polarity_std)
print("Mean: ", polarity_mean)
print("Range: ",polarity_range)

Min:  0.04476190476190476
Max:  0.04476190476190476
Standard Deviation:  0.0
Mean:  0.04476190476190476
Range:  0.0


### TextBlob: Subjectivity analysis

In [85]:
subjectivity_score = []

for phrase in text:
    subjectivity_score.append(TextBlob(phrase).sentiment.subjectivity)

In [86]:
#subjectivity_score

In [87]:
subjectivity_min = min(subjectivity_score)
subjectivity_max = max(subjectivity_score)
subjectivity_std = np.std(subjectivity_score)
subjectivity_mean = np.mean(subjectivity_score)
subjectivity_range = subjectivity_max - subjectivity_min

In [149]:
print("Min: ",subjectivity_min)
print("Max: ",subjectivity_max)
print("Standard Deviation: ", subjectivity_std)
print("median: ", np.median(subjectivity_score))
print("Mean: ", subjectivity_mean)
print("Range: ",subjectivity_range)

Min:  0.535357142857143
Max:  0.535357142857143
Standard Deviation:  0.0
median:  0.535357142857143
Mean:  0.535357142857143
Range:  0.0


### Sentiment Analysis with LSTM

In [89]:
df = pd.read_csv('./data/articles.csv')
df

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
49995,53287,73465,"Rex Tillerson Says Climate Change Is Real, but …",Atlantic,Robinson Meyer,2017-01-11,2017.0,1.0,NaN,"As chairman and CEO of ExxonMobil, Rex Tillers..."
49996,53288,73466,The Biggest Intelligence Questions Raised by t...,Atlantic,Amy Zegart,2017-01-11,2017.0,1.0,NaN,I’ve spent nearly 20 years looking at intellig...
49997,53289,73467,Trump Announces Plan That Does Little to Resol...,Atlantic,Jeremy Venook,2017-01-11,2017.0,1.0,NaN,Donald Trump will not be taking necessary st...
49998,53290,73468,Dozens of For-Profit Colleges Could Soon Close,Atlantic,Emily DeRuy,2017-01-11,2017.0,1.0,NaN,Dozens of colleges could be forced to close ...


In [107]:
# Function to analyze sentiment and return sentiment label
def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    
    if polarity > 0.10316472712094597:
        return 'positive'
    elif polarity < 0.03997114079477093:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis function to the 'content' column and create a new 'sentiment' column
df['sentiment'] = df['content'].apply(get_sentiment)

In [106]:
# text = np.array(df['content'])

# polarity_score = []

# for phrase in text:
#     polarity_score.append(TextBlob(phrase).sentiment.polarity)

tlow = np.percentile(polarity_score, 33)
thigh = np.percentile(polarity_score, 66)
print(tlow)
print(thigh)


0.03997114079477093
0.10316472712094597


In [108]:
df.groupby('sentiment')['sentiment'].value_counts()

sentiment
negative    16500
neutral     16500
positive    17000
Name: count, dtype: int64

In [109]:
def cleaning(df, stop_words):
    df['content'] = df['content'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Replacing the digits/numbers
    df['content'] = df['content'].str.replace('d', '')
    # Removing stop words
    df['content'] = df['content'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['content'] = df['content'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

stop_words = stopwords.words('english')
data_cleaned = cleaning(df, stop_words)

In [141]:
data_cleaned[['content', 'sentiment']]

,content,sentiment
0,washington — congressional republican new fear...,negative
1,"bullet shell get counte, bloo ries votive canl...",negative
2,"walt isney’s “bambi” opene 1942, critic praise...",neutral
3,"eath may great equalizer, isn’t necessarily ev...",positive
4,"seoul, south korea — north korea’s leaer, kim ...",positive
...,...,...
49995,"chairman ceo exxonmobil, rex tillerson amitte ...",neutral
49996,i’ve spent nearly 20 year looking intelligence...,neutral
49997,onal trump taking necessary step resolve confl...,neutral
49998,ozens college coul force close next several ye...,positive


In [146]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(df['content'])
print(text_counts)
#Splitting the data into trainig and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df['sentiment'], test_size=0.25, random_state=5)
#Training the model
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)
#Caluclating the accuracy score of the model
from sklearn import metrics
predicted = MNB.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, Y_test)
print("Accuracuy Score: ",accuracy_score)

c:\Users\simon\Anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  (0, 155343)	1
  (0, 33673)	3
  (0, 119078)	16
  (0, 97437)	2
  (0, 46616)	1
  (0, 32731)	3
  (0, 59503)	11
  (0, 26645)	8
  (0, 79959)	2
  (0, 100450)	5
  (0, 9149)	13
  (0, 157368)	1
  (0, 66034)	1
  (0, 146670)	9
  (0, 35254)	8
  (0, 30051)	2
  (0, 82922)	1
  (0, 38858)	1
  (0, 44730)	5
  (0, 22754)	5
  (0, 137845)	2
  (0, 28561)	1
  (0, 123404)	7
  (0, 14085)	2
  (0, 134193)	1
  :	:
  (49999, 13254)	2
  (49999, 143006)	1
  (49999, 69076)	1
  (49999, 116226)	1
  (49999, 82639)	1
  (49999, 102662)	1
  (49999, 141589)	1
  (49999, 68421)	1
  (49999, 155163)	1
  (49999, 89934)	1
  (49999, 46247)	1
  (49999, 51493)	13
  (49999, 29698)	1
  (49999, 134339)	1
  (49999, 90916)	6
  (49999, 152652)	1
  (49999, 67501)	1
  (49999, 25665)	1
  (49999, 84693)	2
  (49999, 55660)	2
  (49999, 131916)	1
  (49999, 150132)	1
  (49999, 130878)	1
  (49999, 123768)	5
  (49999, 51492)	1
Accuracuy Score:  0.53304


In [148]:
# Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string

# Tokenizer with alphanumeric pattern
token = RegexpTokenizer(r'\b[A-Za-z0-9]+\b')

# Custom function for text preprocessing
def preprocess_text(text):
    # Tokenize and remove stopwords and punctuation
    tokens = token.tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in string.punctuation]
    return ' '.join(tokens)

# Apply preprocessing to 'content' column
df['cleaned_content'] = df['content'].apply(preprocess_text)

# Vectorize text using CountVectorizer
cv = CountVectorizer(ngram_range=(1, 1))  # You can adjust ngram_range as needed
text_counts = cv.fit_transform(df['cleaned_content'])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(text_counts, df['sentiment'], test_size=0.25, random_state=2)

# Training the Multinomial Naive Bayes model
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

# Predicting and evaluating the model
predicted = mnb.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, y_test)
print("Accuracy Score:", accuracy_score)

Accuracy Score: 0.53312
